In [48]:
import os
import sys

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Data Load & Preprocess

In [3]:
vdjdb_raw = pd.read_csv('data/raw/vdjdb.tsv', sep='\t')
iedb_raw = pd.read_csv('data/raw/iedb.csv')
tcr_train = pd.read_csv('data/net_tcr/train_ab_95_alphabeta.csv')

/var/folders/w1/mbcsphtx2xz1xqrls0v3d6r80000gn/T/ipykernel_69657/1976341293.py:2: DtypeWarning: Columns (8,10,12,16,17,18,19,20,21,22,23,24,26,31,32,37,38,45,46,47,48,49,50,51,52,53,60,61,66,67) have mixed types. Specify dtype option on import or set low_memory=False.
  iedb_raw = pd.read_csv('data/raw/iedb.csv')


In [ ]:
iedb_a_merge_curated = pd.merge(tcr_train['CDR3a'], iedb_a, how='inner', left_on='CDR3a', right_on='Chain 1 CDR3 Curated')
iedb_a_merge_calculated = pd.merge(tcr_train['CDR3a'], iedb_a, how='inner', left_on='CDR3a', right_on='Chain 1 CDR3 Calculated')
print('CDR3a IEDB curated:', iedb_a_merge_curated.shape)
print('CDR3a IEDB calculated:', iedb_a_merge_calculated.shape)

iedb_b_merge_curated = pd.merge(tcr_train['CDR3b'], iedb_b, how='inner', left_on='CDR3b', right_on='Chain 2 CDR3 Curated')
iedb_b_merge_calculated = pd.merge(tcr_train['CDR3b'], iedb_b, how='inner', left_on='CDR3b', right_on='Chain 2 CDR3 Calculated')
print('CDR3b IEDB curated:', iedb_b_merge_curated.shape)
print('CDR3b IEDB calculated:', iedb_b_merge_calculated.shape)


In Net-TCR 2.0 paper:
3859 CDRab chains 279 epitopes


## VDJdb

In [4]:
print(vdjdb_raw.columns)
print(vdjdb_raw.shape)

Index(['complex.id', 'Gene', 'CDR3', 'V', 'J', 'Species', 'MHC A', 'MHC B',
       'MHC class', 'Epitope', 'Epitope gene', 'Epitope species', 'Reference',
       'Method', 'Meta', 'CDR3fix', 'Score'],
      dtype='object')
(58446, 17)


In [5]:
vdj_cols = ['Gene', 'CDR3', 'V', 'J', 'Species', 'MHC A', 'MHC B',
       'MHC class']
vdjdb = vdjdb_raw[vdj_cols].copy()

In [6]:
vdjdb['CDR3'].isna().sum()

0

In [7]:
# strip 1st and last AA from chain for lookup
vdjdb['CDR3 strip'] = vdjdb['CDR3'].apply(lambda x: x[1:-1])

# remove duplicates, keep last
vdjdb = vdjdb.drop_duplicates(subset=['Gene', 'CDR3'], keep='last')
vdjdb = vdjdb.astype('str')

In [8]:
vdjdb

,Gene,CDR3,V,J,Species,MHC A,MHC B,MHC class,CDR3 strip
2,TRA,CAYRPPGTYKYIF,TRAV38-2/DV8*01,TRAJ40*01,HomoSapiens,HLA-B*08,B2M,MHCI,AYRPPGTYKYI
3,TRB,CASSALASLNEQFF,TRBV14*01,TRBJ2-1*01,HomoSapiens,HLA-B*08,B2M,MHCI,ASSALASLNEQF
8,TRA,CIVRAPGRADMRF,TRAV26-1*01,TRAJ43*01,HomoSapiens,HLA-B*08,B2M,MHCI,IVRAPGRADMR
9,TRA,CAVPSGAGSYQLTF,TRAV20*01,TRAJ28*01,HomoSapiens,HLA-B*08,B2M,MHCI,AVPSGAGSYQLT
10,TRA,CAYTVLGNEKLTF,TRAV38-1*01,TRAJ48*01,HomoSapiens,HLA-A*02,B2M,MHCI,AYTVLGNEKLT
...,...,...,...,...,...,...,...,...,...
58440,TRB,CASGSGGEHF,TRBV9*01,TRBJ2-7*01,HomoSapiens,HLA-B*07:02,B2M,MHCI,ASGSGGEH
58441,TRA,CAVVGFSDGQKLLF,TRAV8-3*01,nan,HomoSapiens,HLA-B*07:02,B2M,MHCI,AVVGFSDGQKLL
58442,TRB,CASSELDGRL,TRBV2*01,TRBJ2-1*01,HomoSapiens,HLA-B*07:02,B2M,MHCI,ASSELDGR
58444,TRB,CASSPTDNF,TRBV9*01,TRBJ2-3*01,HomoSapiens,HLA-B*07:02,B2M,MHCI,ASSPTDN


In [9]:
# split into alpha and beta tables
vdjdb_a = vdjdb[vdjdb['Gene'] == 'TRA']
vdjdb_b = vdjdb[vdjdb['Gene'] == 'TRB']

print('VDJdb CDR3a:', vdjdb_a.shape)
print('VDJdb CDR3b:', vdjdb_b.shape)


VDJdb CDR3a: (20702, 9)
VDJdb CDR3b: (22087, 9)


In [66]:
## Convert VDJdb into CDR3 alpha and beta dictionaries

vdjdb_a_idx = vdjdb_a.drop_duplicates(subset='CDR3 strip', keep='last')
vdjdb_b_idx = vdjdb_b.drop_duplicates(subset='CDR3 strip', keep='last')

vdjdb_a_idx = vdjdb_a_idx.set_index('CDR3 strip', drop=True)
vdjdb_b_idx = vdjdb_b_idx.set_index('CDR3 strip', drop=True)

vdjdb_a_dict = vdjdb_a_idx.to_dict('index')
vdjdb_b_dict = vdjdb_b_idx.to_dict('index')

print('CDR3a unique sequences:', len(vdjdb_a_dict.keys()))
print('CDR3b unique sequences:', len(vdjdb_b_dict.keys()))

CDR3a unique sequences: 20702
CDR3b unique sequences: 22086


In [70]:
tcr_add_cols = ['CDR3a', 'CDR3b', 'peptide', 'partition', 'binder']
vdj_add_cols = ['cdra', 'va', 'ja', 'cdrb', 'vb', 'jb', 'species', 'mhc', 'mhc_class']
tcr_train_geom = tcr_train.reindex(columns=tcr_add_cols+vdj_add_cols)
tcr_train_geom

,CDR3a,CDR3b,peptide,partition,binder,cdra,va,ja,cdrb,vb,jb,species,mhc,mhc_class
0,ADLQTNARLM,SGGQGMVDGYT,NLVPMVATV,5,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AARANNARLM,ASSLEYSPRPYEQY,NLVPMVATV,5,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AGGGSSNTGKLI,ASSLLLAGYNEQF,NLVPMVATV,5,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AVRGTNARLM,ASSVVTEAF,NLVPMVATV,5,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AVRDTNARLM,ASSVVTEAF,NLVPMVATV,5,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16459,AVHRVGADKLI,ASSLGVWASGTDTQY,GILGFVFTL,5,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16460,AASGPGANNLF,ASGRVGQETQY,GILGFVFTL,5,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16461,AMKSTNAGKST,ASSVGQGVRSF,GILGFVFTL,5,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16462,AGLITQGGSEKLV,SVSRGPYEQY,GILGFVFTL,5,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [71]:
## Search for CDR3 chains in TCR training data

vdjdb_absent = {'cdr3a': list(), 'cdr3b': list()}
for idx in tcr_train_geom.index:
    try:
        vdj_data_a = vdjdb_a_dict[tcr_train_geom.iloc[idx]['CDR3a']]
        vdj_data_b = vdjdb_b_dict[tcr_train_geom.iloc[idx]['CDR3b']]
        assert vdj_data_a['Species'] == vdj_data_b['Species'] # check that species match
        assert vdj_data_a['MHC A'] == vdj_data_b['MHC A'] # check that HLA structures match
        assert vdj_data_a['MHC class'] == vdj_data_b['MHC class'] # check that MHC classes match
        tcr_train_geom.loc[idx, 'cdra'] = vdj_data_a['CDR3']
        tcr_train_geom.loc[idx, 'va'] = vdj_data_a['V']
        tcr_train_geom.loc[idx, 'ja'] = vdj_data_a['J']
        tcr_train_geom.loc[idx, 'cdrb'] = vdj_data_b['CDR3']
        tcr_train_geom.loc[idx, 'vb'] = vdj_data_b['V']
        tcr_train_geom.loc[idx, 'jb'] = vdj_data_b['J']
        tcr_train_geom.loc[idx, 'species'] = vdj_data_a['Species']
        tcr_train_geom.loc[idx, 'mhc'] = vdj_data_a['MHC A']
        tcr_train_geom.loc[idx, 'mhc_class'] = vdj_data_a['MHC class']
    except:
        vdjdb_absent['cdr3a'].append(tcr_train_geom.iloc[idx]['CDR3a'])
        vdjdb_absent['cdr3b'].append(tcr_train_geom.iloc[idx]['CDR3b'])
        
print('cdr3a absent:', len(vdjdb_absent['cdr3a']))
print('cdr3b absent:', len(vdjdb_absent['cdr3b']))


cdr3a absent: 12518
cdr3b absent: 12518


In [72]:
tcr_train_geom

,CDR3a,CDR3b,peptide,partition,binder,cdra,va,ja,cdrb,vb,jb,species,mhc,mhc_class
0,ADLQTNARLM,SGGQGMVDGYT,NLVPMVATV,5,1,CADLQTNARLMF,TRAV5*01,TRAJ31*01,CSGGQGMVDGYTF,TRBV29-1*01,TRBJ1-2*01,HomoSapiens,HLA-A*02:01,MHCI
1,AARANNARLM,ASSLEYSPRPYEQY,NLVPMVATV,5,1,CAARANNARLMF,TRAV29/DV5*01,TRAJ31*01,CASSLEYSPRPYEQYF,TRBV7-9*01,TRBJ2-7*01,HomoSapiens,HLA-A*02:01,MHCI
2,AGGGSSNTGKLI,ASSLLLAGYNEQF,NLVPMVATV,5,1,CAGGGSSNTGKLIF,TRAV8-3*01,TRAJ37*01,CASSLLLAGYNEQFF,TRBV11-2*01,TRBJ2-1*01,HomoSapiens,HLA-A*02:01,MHCI
3,AVRGTNARLM,ASSVVTEAF,NLVPMVATV,5,1,CAVRGTNARLMF,TRAV3*01,TRAJ31*01,CASSVVTEAFF,TRBV12-3*01,TRBJ1-1*01,HomoSapiens,HLA-A*02:01,MHCI
4,AVRDTNARLM,ASSVVTEAF,NLVPMVATV,5,1,CAVRDTNARLMF,TRAV3*01,TRAJ31*01,CASSVVTEAFF,TRBV12-3*01,TRBJ1-1*01,HomoSapiens,HLA-A*02:01,MHCI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16459,AVHRVGADKLI,ASSLGVWASGTDTQY,GILGFVFTL,5,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16460,AASGPGANNLF,ASGRVGQETQY,GILGFVFTL,5,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16461,AMKSTNAGKST,ASSVGQGVRSF,GILGFVFTL,5,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16462,AGLITQGGSEKLV,SVSRGPYEQY,GILGFVFTL,5,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## IEDB

In [51]:
print(iedb_raw.columns)
print(iedb_raw.shape)

Index(['Group Receptor ID', 'Receptor ID', 'Reference IRI', 'Epitope IRI',
       'Description', 'Antigen', 'Organism', 'Response Type', 'Assay IDs',
       'MHC Allele Names', 'Reference Name', 'STABLE ID', 'Synonyms',
       'Receptor Type', 'Chain 1 Type', 'Chain 1 Species',
       'Chain 1 Nucleotide', 'Curated Chain 1 V Gene',
       'Calculated Chain 1 V Gene', 'Curated Chain 1 D Gene',
       'Calculated Chain 1 D Gene', 'Curated Chain 1 J Gene',
       'Calculated Chain 1 J Gene', 'Chain 1 Full Sequence',
       'Chain 1 Accession', 'Chain 1 CDR3 Curated', 'Chain 1 CDR3 Calculated',
       'Chain 1 CDR3 Start Curated', 'Chain 1 CDR3 End Curated',
       'Chain 1 CDR3 Start Calculated', 'Chain 1 CDR3 End Calculated',
       'Chain 1 CDR1 Curated', 'Chain 1 CDR1 Calculated',
       'Chain 1 CDR1 Start Curated', 'Chain 1 CDR1 End Curated',
       'Chain 1 CDR1 Start Calculated', 'Chain 1 CDR1 End Calculated',
       'Chain 1 CDR2 Curated', 'Chain 1 CDR2 Calculated',
       'Chain 

In [55]:
subset = ['Chain 1 CDR3 Curated', 'Chain 2 CDR3 Curated']
iedb = iedb_raw.dropna(subset=subset)
iedb = iedb.astype('str')
print(iedb.shape)

(22771, 72)


In [61]:
iedb_base_cols = ['Group Receptor ID', 'Receptor ID', 'Reference IRI', 'Epitope IRI',
       'Description', 'Antigen', 'Organism', 'Response Type', 'Assay IDs',
       'MHC Allele Names', 'Reference Name', 'STABLE ID', 'Synonyms',
       'Receptor Type']

iedb_a_cols = [col for col in iedb.columns if "Chain 1" in col]
iedb_b_cols = [col for col in iedb.columns if "Chain 2" in col]

iedb_a = iedb[iedb_base_cols + iedb_a_cols]
iedb_b = iedb[iedb_base_cols + iedb_b_cols]

print(iedb_a.shape)
print(iedb_b.shape)

(22771, 43)
(22771, 43)


In [75]:
iedb_a_idx = iedb_a.drop_duplicates(subset='Chain 1 CDR3 Curated', keep='last')
iedb_b_idx = iedb_b.drop_duplicates(subset='Chain 2 CDR3 Curated', keep='last')

iedb_a_idx = iedb_a_idx.set_index('Chain 1 CDR3 Curated', drop=True)
iedb_b_idx = iedb_b_idx.set_index('Chain 2 CDR3 Curated', drop=True)

iedb_a_dict = iedb_a_idx.to_dict('index')
iedb_b_dict = iedb_b_idx.to_dict('index')

print('CDR3a unique sequences:', len(iedb_a_dict.keys()))
print('CDR3b unique sequences:', len(iedb_b_dict.keys()))

CDR3a unique sequences: 17054
CDR3b unique sequences: 18051


In [77]:
iedb_absent = {'cdr3a': list(), 'cdr3b': list()}
for idx in tcr_train_geom.index:
    try:
        iedb_data_a = iedb_a_dict[tcr_train_geom.iloc[idx]['CDR3a']]
        iedb_data_b = iedb_b_dict[tcr_train_geom.iloc[idx]['CDR3b']]
        assert iedb_data_a['Organism'] == iedb_data_b['Organism'] # check that species match
        assert iedb_data_a['MHC Allele Names'] == iedb_data_b['MHC Allele Names'] # check that HLA structures match
        # assert iedb_data_a['MHC class'] == iedb_data_b['MHC class'] # check that MHC classes match
        tcr_train_geom.loc[idx, 'cdra'] = iedb_data_a['Chain 1 CDR3 Curated']
        tcr_train_geom.loc[idx, 'va'] = iedb_data_a['Curated Chain 1 V Gene']
        tcr_train_geom.loc[idx, 'ja'] = iedb_data_a['Curated Chain 1 J Gene']
        tcr_train_geom.loc[idx, 'cdrb'] = iedb_data_b['Chain 2 CDR3 Curated']
        tcr_train_geom.loc[idx, 'vb'] = iedb_data_b['Curated Chain 2 V Gene']
        tcr_train_geom.loc[idx, 'jb'] = iedb_data_b['Curated Chain 2 J Gene']
        tcr_train_geom.loc[idx, 'species'] = iedb_data_a['Organism']
        tcr_train_geom.loc[idx, 'mhca'] = iedb_data_a['MHC Allele Names']
        # tcr_train_geom.loc[idx, 'mhcb'] = iedb_data_a['MHC B']
        # tcr_train_geom.loc[idx, 'mhc_class'] = iedb_data_a['MHC class']
    except:
        iedb_absent['cdr3a'].append(tcr_train_geom.iloc[idx]['CDR3a'])
        iedb_absent['cdr3b'].append(tcr_train_geom.iloc[idx]['CDR3b'])
        
print('cdr3a absent:', len(iedb_absent['cdr3a']))
print('cdr3b absent:', len(iedb_absent['cdr3b']))


cdr3a absent: 16464
cdr3b absent: 16464


In [78]:
tcr_train_geom

,CDR3a,CDR3b,peptide,partition,binder,cdra,va,ja,cdrb,vb,jb,species,mhc,mhc_class
0,ADLQTNARLM,SGGQGMVDGYT,NLVPMVATV,5,1,CADLQTNARLMF,TRAV5*01,TRAJ31*01,CSGGQGMVDGYTF,TRBV29-1*01,TRBJ1-2*01,HomoSapiens,HLA-A*02:01,MHCI
1,AARANNARLM,ASSLEYSPRPYEQY,NLVPMVATV,5,1,CAARANNARLMF,TRAV29/DV5*01,TRAJ31*01,CASSLEYSPRPYEQYF,TRBV7-9*01,TRBJ2-7*01,HomoSapiens,HLA-A*02:01,MHCI
2,AGGGSSNTGKLI,ASSLLLAGYNEQF,NLVPMVATV,5,1,CAGGGSSNTGKLIF,TRAV8-3*01,TRAJ37*01,CASSLLLAGYNEQFF,TRBV11-2*01,TRBJ2-1*01,HomoSapiens,HLA-A*02:01,MHCI
3,AVRGTNARLM,ASSVVTEAF,NLVPMVATV,5,1,CAVRGTNARLMF,TRAV3*01,TRAJ31*01,CASSVVTEAFF,TRBV12-3*01,TRBJ1-1*01,HomoSapiens,HLA-A*02:01,MHCI
4,AVRDTNARLM,ASSVVTEAF,NLVPMVATV,5,1,CAVRDTNARLMF,TRAV3*01,TRAJ31*01,CASSVVTEAFF,TRBV12-3*01,TRBJ1-1*01,HomoSapiens,HLA-A*02:01,MHCI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16459,AVHRVGADKLI,ASSLGVWASGTDTQY,GILGFVFTL,5,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16460,AASGPGANNLF,ASGRVGQETQY,GILGFVFTL,5,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16461,AMKSTNAGKST,ASSVGQGVRSF,GILGFVFTL,5,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16462,AGLITQGGSEKLV,SVSRGPYEQY,GILGFVFTL,5,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Merge VDJ to Net TCR

In [ ]:
vdj_a_merge = pd.merge(tcr_train, vdjdb_a, how='left', left_on='CDR3a', right_on='CDR3')
vdj_a_merge_strip = pd.merge(tcr_train, vdjdb_a, how='left', left_on='CDR3a', right_on='CDR3 strip')
print('CDR3a VDJdb:', vdj_a_merge.shape)
print('CDR3a VDJdb stripped:', vdj_a_merge_strip.shape)

vdj_b_merge = pd.merge(tcr_train, vdjdb_b, how='left', left_on='CDR3b', right_on='CDR3')
vdj_b_merge_strip = pd.merge(tcr_train, vdjdb_b, how='left', left_on='CDR3b', right_on='CDR3 strip')
print('CDR3b VDJdb:', vdj_b_merge.shape)
print('CDR3b VDJdb stripped:', vdj_b_merge_strip.shape)



In [ ]:
vdj_merge = pd.merge(vdj_a_merge_strip, vdj_b_merge_strip, how='inner', on=['CDR3a', 'CDR3b'])
vdj_merge

In [ ]:
vdj_merge = vdj_merge.drop_duplicates(subset=['CDR3_x', 'CDR3_y'], keep='last')
assert (vdj_merge['partition_x']==vdj_merge['partition_y']).all()
assert (vdj_merge['binder_x']==vdj_merge['binder_y']).all()
assert (vdj_merge['peptide_x']==vdj_merge['peptide_y']).all()
vdj_merge